In [32]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras

import os
from dotenv import load_dotenv
load_dotenv()


True

In [33]:
def nan_to_null(f,
        _NULL=psycopg2.extensions.AsIs('NULL'),
        _Float=psycopg2.extensions.Float):
    if not np.isnan(f):
        return _Float(f)
    return _NULL

psycopg2.extensions.register_adapter(float, nan_to_null)

In [34]:
conn = psycopg2.connect(
    host=os.environ['POSTGRES_HOST'],
    port=os.environ['POSTGRES_PORT'],
    database=os.environ['OLAP_DB'],
    user=os.environ['POSTGRES_USER'],
    password=os.environ['POSTGRES_PASSWORD']
)

In [35]:
def execute_values(conn, df, table): 
  
    tuples = [tuple(x) for x in df.to_numpy()] 
  
    cols = ','.join(list(df.columns)) 
    # SQL query to execute 
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols) 
    cursor = conn.cursor() 
    try: 
        extras.execute_values(cursor, query, tuples) 
        conn.commit() 
    except (Exception, psycopg2.DatabaseError) as error: 
        print("Error: %s" % error) 
        conn.rollback() 
        cursor.close() 
        return 1
    cursor.close() 

# Upload properties

In [ ]:
final = pd.read_csv('../data/processed/20231116/processed.csv')
final.info()

In [40]:
columns = [
    'id','name','url','price','num_bedroom','num_bathroom','cost_psf',
    'address','road_name','building','postal_code','latitude','longitude',
    'floor_area','land_area','walk_destination','walk_distance_m','walk_time_mins',
    'lease_duration','completion','type',
]
properties = final[columns]

execute_values(conn, properties, 'properties')

# Upload Amenities

In [53]:
final = pd.read_csv('../data/processed/20231116/property_amenities.csv')
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22758189 entries, 0 to 22758188
Data columns (total 6 columns):
 #   Column        Dtype  
---  ------        -----  
 0   property_id   int64  
 1   amenity_type  object 
 2   amenity_name  object 
 3   latitude      float64
 4   longitude     float64
 5   distance_km   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 1.0+ GB


In [46]:
columns = [
    'property_id', 'amenity_type', 'amenity_name','latitude','longitude','distance_km'
]
properties_amenities = final[columns]

execute_values(conn, properties_amenities, 'property_amenities')